In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
ga_path = r'C:\Users\huda\Downloads\ga_worker_results\ga_worker_results_rewritten'
file_paths = [r'C:\Users\huda\Downloads\results_ortools.txt', r'C:\Users\huda\Downloads\results_cplex_cp.txt', r'C:\Users\huda\Downloads\results_gurobi.txt', r'C:\Users\huda\Downloads\results_hexaly_rewritten.txt', r'C:\Users\huda\Downloads\results_cplex_lp_rewritten.txt']
optimizers = ['ORTOOLS', 'CPLEX-CP', 'GUROBI', 'HEXALY', 'CPLEX-LP']

In [3]:
def read_file(file : str) -> pd.DataFrame:
    file_content = pd.read_csv(file, names=['name','optimization_status','fitness_value','lower_bound', 'runtime', 'result_vector1', 'result_vector2', 'result_vector3', 'peak_cpu', 'peak_ram', 'resource_history', 'best_result_history'], sep=';')#, converters={'best_result_history': pd.eval})
    # NOTE: fill optimization status for GA versions with 0
    #file_content = pd.read_csv(file, names=['name','optimization_status','fitness_value', 'runtime', 'resource_history', 'best_result_history'], sep=';')
    df = pd.DataFrame(file_content)
    del file_content
    df = df.drop(columns=['result_vector1', 'result_vector2', 'result_vector3', 'peak_cpu', 'peak_ram'])
    df = df[~df['name'].str.startswith('Error')]
    df = df[df['fitness_value'] != float('inf')]
    return df

In [4]:
def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[0].startswith('Behnke'):
        return 'Behnke'+values[-1]
    if values[0].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[-1]
    if values[0].startswith('Chambers'):
        return 'ChambersBarnes'+values[-1]
    if values[0].startswith('Hurink') and values[1].startswith('s'):
        return 'HurinkSdata'+values[-1]
    if values[0].startswith('Hurink') and values[1].startswith('e'):
        return 'HurinkEdata'+values[-1]
    if values[0].startswith('Hurink') and values[1].startswith('r'):
        return 'HurinkRdata'+values[-1]
    if values[0].startswith('Hurink') and values[1].startswith('v'):
        return 'HurinkVdata'+values[-1]
    if values[0].startswith('DP'):
        return 'DPpaulli'+values[-1]
    if values[0].startswith('Kacem'):
        return 'Kacem'+values[-1]
    if values[0].startswith('Fattahi'):
        return 'Fattahi'+values[-1]
    return name

In [5]:
def extract_solver_data(fjssp_w):
    compare = dict()
    for index, row in fjssp_w.iterrows():
        name = remap(row['name'][2:-12])
        if name not in compare:
            compare[name] = {
                'fitness': row['fitness_value'],
                'lower_bound': row['lower_bound'],
                'status': row['optimization_status'],
                'runtime': row['runtime']
            }
    return compare

In [6]:
cp_fjssp_w = read_file(file_paths[1])
cplp_fjssp_w = read_file(file_paths[4])
gurobi_fjssp_w = read_file(file_paths[2])
ortools_fjssp_w = read_file(file_paths[3])
solver = dict()
solver['cplex_cp'] = extract_solver_data(cp_fjssp_w)
solver['cplex_lp'] = extract_solver_data(cplp_fjssp_w)
solver['gurobi'] = extract_solver_data(gurobi_fjssp_w)
solver['ortools'] = extract_solver_data(ortools_fjssp_w)

In [7]:
ga_fjssp_w_data = dict()
files = os.listdir(ga_path)
for file in files:
    with open(ga_path + '/' + file, 'r') as f:
        json_data = json.loads(f.read())
        ga_fjssp_w_data[remap(file[2:-17])] = json_data

In [8]:
ga_average_fjssp = dict()
ga_average_fjssp_w = dict()
ga_best_fjssp = dict()
ga_best_fjssp_w = dict()
for instance in ga_fjssp_w_data:
    ga_best_fjssp_w[instance] = float('inf')
    ga_average_fjssp_w[instance] = 0
    for run in ga_fjssp_w_data[instance]['results']:
        fitness = run['Result']['Fitness']['Makespan']
        if fitness < ga_best_fjssp_w[instance]:
            ga_best_fjssp_w[instance] = fitness
        ga_average_fjssp_w[instance] += fitness
    ga_average_fjssp_w[instance] /= len( ga_fjssp_w_data[instance]['results'])
    

In [9]:
compare = dict()
for instance in ga_average_fjssp_w:
    compare[instance] = {
        'fitness': ga_average_fjssp_w[instance],
        #'runtime': row['runtime'] # TODO: maybe?
    }
solver['ga_average'] = compare
compare = dict()
for instance in ga_best_fjssp_w:
    compare[instance] = {
        'fitness': ga_best_fjssp_w[instance],
        #'runtime': row['runtime'] # TODO: maybe?
    }
solver['ga_best'] = compare

In [10]:
solver['ga_best'].keys()

dict_keys(['Behnke10', 'Behnke11', 'Behnke12', 'Behnke13', 'Behnke14', 'Behnke15', 'Behnke16', 'Behnke17', 'Behnke18', 'Behnke19', 'Behnke1', 'Behnke20', 'Behnke21', 'Behnke22', 'Behnke23', 'Behnke24', 'Behnke25', 'Behnke26', 'Behnke27', 'Behnke28', 'Behnke29', 'Behnke2', 'Behnke30', 'Behnke31', 'Behnke32', 'Behnke33', 'Behnke34', 'Behnke35', 'Behnke36', 'Behnke37', 'Behnke38', 'Behnke39', 'Behnke3', 'Behnke40', 'Behnke41', 'Behnke42', 'Behnke43', 'Behnke44', 'Behnke45', 'Behnke46', 'Behnke47', 'Behnke48', 'Behnke49', 'Behnke4', 'Behnke50', 'Behnke51', 'Behnke52', 'Behnke53', 'Behnke54', 'Behnke55', 'Behnke56', 'Behnke57', 'Behnke58', 'Behnke59', 'Behnke5', 'Behnke60', 'Behnke6', 'Behnke7', 'Behnke8', 'Behnke9', 'BrandimarteMk10', 'BrandimarteMk11', 'BrandimarteMk12', 'BrandimarteMk13', 'BrandimarteMk14', 'BrandimarteMk15', 'BrandimarteMk1', 'BrandimarteMk2', 'BrandimarteMk3', 'BrandimarteMk4', 'BrandimarteMk5', 'BrandimarteMk6', 'BrandimarteMk7', 'BrandimarteMk8', 'BrandimarteMk9', 'H

In [11]:
solver['cplex_cp'].keys()

dict_keys(['Behnke10', 'Behnke11', 'Behnke12', 'Behnke13', 'Behnke14', 'Behnke15', 'Behnke16', 'Behnke17', 'Behnke18', 'Behnke19', 'Behnke1', 'Behnke20', 'Behnke21', 'Behnke22', 'Behnke23', 'Behnke24', 'Behnke25', 'Behnke26', 'Behnke27', 'Behnke28', 'Behnke29', 'Behnke2', 'Behnke30', 'Behnke31', 'Behnke32', 'Behnke33', 'Behnke34', 'Behnke35', 'Behnke36', 'Behnke37', 'Behnke38', 'Behnke39', 'Behnke3', 'Behnke40', 'Behnke41', 'Behnke42', 'Behnke43', 'Behnke44', 'Behnke45', 'Behnke46', 'Behnke47', 'Behnke48', 'Behnke49', 'Behnke4', 'Behnke50', 'Behnke51', 'Behnke52', 'Behnke53', 'Behnke54', 'Behnke55', 'Behnke56', 'Behnke57', 'Behnke58', 'Behnke59', 'Behnke5', 'Behnke60', 'Behnke6', 'Behnke7', 'Behnke8', 'Behnke9', 'BrandimarteMk11', 'BrandimarteMk12', 'BrandimarteMk13', 'BrandimarteMk14', 'BrandimarteMk15', 'BrandimarteMk1', 'BrandimarteMk2', 'BrandimarteMk3', 'BrandimarteMk4', 'BrandimarteMk5', 'BrandimarteMk6', 'BrandimarteMk7', 'BrandimarteMk9', 'HurinkSdata10', 'HurinkSdata11', 'Huri

In [15]:
best = dict()
for instance in solver['ga_best']:
    best[instance] = float('inf')
    for s in solver:
        if instance in solver[s]:
            if solver[s][instance]['fitness'] < best[instance]:
                best[instance] = solver[s][instance]['fitness']


In [16]:
def calculate_gap(fitness, best):
    return ((fitness - best) / best)

In [22]:
def get_in_gap(max_gap):
    solver_gap = dict()
    for instance in solver['ga_best']:
        for s in solver:
            if instance in solver[s]:
                gap = calculate_gap(solver[s][instance]['fitness'], best[instance])
                if gap <= max_gap:
                    if s not in solver_gap:
                        solver_gap[s] = []
                    solver_gap[s].append(instance)
    return solver_gap

In [20]:
def show_in_gap(max_gap):
    solver_gap = get_in_gap(max_gap)
    print(max_gap)
    for s in solver_gap:
        print(f'{s}: {len(solver_gap[s])}')


In [24]:
show_in_gap(0)
show_in_gap(0.05)
show_in_gap(0.1)
show_in_gap(0.2)
show_in_gap(0.3)

0
cplex_cp: 346
ga_best: 30
ortools: 58
ga_average: 9
gurobi: 20
cplex_lp: 9
0.05
cplex_cp: 398
gurobi: 99
ortools: 215
ga_best: 164
ga_average: 113
cplex_lp: 27
0.1
cplex_cp: 400
ga_best: 214
gurobi: 158
ortools: 268
ga_average: 182
cplex_lp: 36
0.2
cplex_cp: 400
ga_average: 275
ga_best: 304
gurobi: 220
ortools: 317
cplex_lp: 63
0.3
cplex_cp: 400
ortools: 353
ga_average: 329
ga_best: 335
gurobi: 244
cplex_lp: 104


In [30]:
optimal_instances = []
for instance in solver['ga_best']:
    for s in solver:
        if s == 'cplex_cp' or s == 'ortools':
            if instance in solver[s]:
                if solver[s][instance]['status'] == 1:
                    optimal_instances.append(instance)

In [31]:
len(optimal_instances)

84

In [32]:
84/402

0.208955223880597